__Problem 43__
----
Given the table below, called 'orders', write code to show the average revenue by month by channel:

In [53]:
import pandas as pd
raw_data = {'order_id': [1,2,3,4,5,6],
            'channel': ['Online', 'Online', 'In_store', 'In_store','Online','Online'],
            'date': ['2018-09-01', '2018-09-03', '2018-10-11', "2018-08-21","2018-08-13",'2018-10-29'],
            'month': ['09', '09', '10', '08','08','10'],
            'revenue': [100,125,200,80,200,100]}
df = pd.DataFrame(raw_data, columns = ['order_id', 'channel', 'date','month', 'revenue'])
df

,order_id,channel,date,month,revenue
0,1,Online,2018-09-01,09,100
1,2,Online,2018-09-03,09,125
2,3,In_store,2018-10-11,10,200
3,4,In_store,2018-08-21,08,80
4,5,Online,2018-08-13,08,200
5,6,Online,2018-10-29,10,100


Your result should return the following in a structured table:

<pre>Month | Channel | Avg. Revenue</pre>

_____
Solution

In [2]:
df_new = df.groupby(['month', 'channel'])['revenue'].mean().reset_index()
df_new.rename(columns={'revenue':'avg_revenue'})

,month,channel,avg_revenue
0,08,In_store,80.0
1,08,Online,200.0
2,09,Online,112.5
3,10,In_store,200.0
4,10,Online,100.0


Problem 44 
-----
Suppose you work for a retail company that has a database containing two tables, one called 'orders', and one called 'revenue', as shown below:

In [20]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE Orders (
    order_id INTEGER,
    channel VARCHAR(1024),
    date VARCHAR(1024),
    month VARCHAR(1024)
    );
    
    CREATE TABLE Revenue (
        order_id INTEGER,
        revenue INTEGER
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [21]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO Orders
    (order_id, channel, date, month)
    VALUES
    (1, "Online", "2018-09-01", "September"),
    (2, "Online", "2018-09-03", "September"),
    (3, "In_store", "2018-10-11", "October"),
    (4, "In_store", "2018-08-21", "August"),
    (5, "Online", "2018-08-13", "August"),
    (6, "Online", "2018-10-29", "October");
    
    INSERT or IGNORE INTO Revenue
    (order_id, revenue)
    VALUES
    (1, 100),
    (2, 125),
    (3, 200),
    (4, 80),
    (5, 200),
    (6, 100);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [50]:
def grab_order_table() -> str:
    return """
    SELECT *
    FROM Orders;
    """

crsr.execute(grab_order_table())
orders = crsr.fetchall()

pd.DataFrame(orders, columns=['order_id', 
                              'channel', 
                              'date', 
                              'month'])

,order_id,channel,date,month
0,1,Online,2018-09-01,September
1,2,Online,2018-09-03,September
2,3,In_store,2018-10-11,October
3,4,In_store,2018-08-21,August
4,5,Online,2018-08-13,August
5,6,Online,2018-10-29,October


In [51]:
def grab_revenue_table() -> str:
    return """
    SELECT *
    FROM Revenue;
    """

crsr.execute(grab_revenue_table())
revenue = crsr.fetchall()

pd.DataFrame(revenue, columns=['order_id', 'revenue'])

,order_id,revenue
0,1,100
1,2,125
2,3,200
3,4,80
4,5,200
5,6,100


In [52]:
def get_sql() -> str:
    return """
    SELECT o.channel,
           SUM(r.revenue) AS total_revenue
    FROM Orders o
    JOIN Revenue r
    ON o.order_id = r.order_id
    WHERE o.month IN ("September", "October")
    GROUP BY o.channel
    """
crsr.execute(get_sql2())
result = crsr.fetchall()
pd.DataFrame(result, columns=['channel', 'total_revenue'])

,channel,total_revenue
0,In_store,200
1,Online,325


Problem 42
-----

You are given an array containing reviews for a popular iOS app below:

reviews = [‘app is good, but forced updates are too frequent‘, ‘love this app, use it daily to log calories', 'free version of this app has way too many ads’, ‘app doesn't load, 0/10’]

Your task is to determine sentiment from the review above. To do this you first decide to write code to find the count of individual words across all the reviews -- write this code using Python.

In [1]:
def word_dist(reviews):
    word_count = {}
    
    for review in reviews:
        words = review.split(' ')
        for word in words:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1
    return word_count

In [2]:
reviews = ['app is good, but forced updates are too frequent', 
           'love this app, use it daily to log calories', 
           'free version of this app has way too many ads', 
           "app doesn't load, 0/10"]

word_dist(reviews)

{'app': 3,
 'is': 1,
 'good,': 1,
 'but': 1,
 'forced': 1,
 'updates': 1,
 'are': 1,
 'too': 2,
 'frequent': 1,
 'love': 1,
 'this': 2,
 'app,': 1,
 'use': 1,
 'it': 1,
 'daily': 1,
 'to': 1,
 'log': 1,
 'calories': 1,
 'free': 1,
 'version': 1,
 'of': 1,
 'has': 1,
 'way': 1,
 'many': 1,
 'ads': 1,
 "doesn't": 1,
 'load,': 1,
 '0/10': 1}

Problem 40
----
Given the table below, called 'orders', write a SQL query to show the monthly revenue growth. To calculate the monthly revenue growth, you can apply the following logic:

revenue growth = (current month's revenue-prior month's revenue)/prior month's revenue.



In [48]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE Orders (
    order_id INTEGER,
    channel VARCHAR(1024),
    date VARCHAR(1024),
    month INTEGER,
    revenue FLOAT
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [49]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO Orders
    (order_id, channel, date, month, revenue)
    VALUES
    (1, "Online", "2018-09-01", 9, 100),
    (2, "Online", "2018-09-03", 9, 125),
    (3, "In_store", "2018-10-11", 10, 200),
    (4, "In_store", "2018-08-21", 8, 80),
    (5, "Online", "2018-08-13", 8, 200);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [55]:
def get_sql() -> str:
    return """
    SELECT s.month,
           (s.current_rev-s.prior_rev)/s.prior_rev
    FROM (
        SELECT month,
               SUM(revenue) AS current_rev,
               LAG(SUM(revenue),1) OVER (ORDER BY month) AS prior_rev
        FROM Orders
        GROUP BY month
        ORDER BY month
    ) s
    """
crsr.execute(get_sql())
result = crsr.fetchall()
pd.DataFrame(result, columns=['month', 'grow_rate'])

,month,grow_rate
0,8,NaN
1,9,-0.196429
2,10,-0.111111


Problem 39
----
You are given a dataframe containing student information, named df (shown below).
Suppose you want to normalize each student's grade based on their age.

In [56]:
import pandas as pd

raw_data = {'name': ['Willard Morris', 'Al Jennings', 'Omar Mullins', 'Spencer McDaniel'],
                'age': [20, 19, 22, 21],
                'favorite_color': ['blue', 'blue', 'yellow', "green"],
                'grade': [88, 92, 95, 70]}

df = pd.DataFrame(raw_data)
df.head()

,name,age,favorite_color,grade
0,Willard Morris,20,blue,88
1,Al Jennings,19,blue,92
2,Omar Mullins,22,yellow,95
3,Spencer McDaniel,21,green,70


Write a function using Python Pandas that will add a new column to your dataframe containing a new grade normalized against the mean age of the students.


In [59]:
df['grade_norm'] = df['grade']*(df['age']/df['age'].mean())

In [60]:
df

,name,age,favorite_color,grade,grade_norm
0,Willard Morris,20,blue,88,85.853659
1,Al Jennings,19,blue,92,85.268293
2,Omar Mullins,22,yellow,95,101.951220
3,Spencer McDaniel,21,green,70,71.707317


Problem 38
---------
Suppose an individual is taxed 30% if earnings for a given week are >= \\$2,000. 
If earnings land < \\$2,000 for the week, the individual is taxed at a lower rate of 15%.

Write a function using Python to calculate both the pre-tax and post-tax earnings for a given individual, with the ability to feed in the hourly wage and the weekly hours as inputs.

For example, if an individual earns $55/hour and works for 40 hours, the function should return:

Pre-tax earnings were 55 * 40 = \\$2,200 for the week.
Post-tax earnings were \\$2,200*.7 (since we fall in higher tax bracket here) = \\$1,540 for the week

In [61]:
def income_tax(rate, hrs):
    pre_tax = rate*hrs
    post_tax = pre_tax*(1-0.15)
    
    if pre_tax >= 2000:
        post_tax = pre_tax*(1-0.3)
    
    return pre_tax, post_tax 

In [62]:
income_tax(55, 40)

(2200, 1540.0)

Problem 37
----

Suppose you're trying to understand how many managers you have per employee at Company XYZ. On your search to understand, you are given two tables: (1) managers and (2) employees. Each table has 1 column named id.

Given this dataset, can you use SQL to find the employees that are also managers? Hint: given the table names as well as the single column name you should be able to write a full SQL query.

In [1]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE managers (
        id INTEGER
    );
    
    CREATE TABLE employees (
        id INTEGER
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [2]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO managers
    (id)
    VALUES
    (1),
    (2),
    (3),
    (4),
    (5),
    (6);
    
    INSERT or IGNORE INTO employees
    (id)
    VALUES
    (1),
    (2),
    (3),
    (4),
    (5),
    (6),
    (40),
    (23),
    (21),
    (400);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [3]:
def get_sql() -> str:
    return """
    SELECT e.*
    FROM employees e
    JOIN managers m
    ON e.id = m.id
    """

crsr.execute(get_sql())
result = crsr.fetchall()

In [4]:
result

[(1,), (2,), (3,), (4,), (5,), (6,)]

Problem 36
--- 
Suppose you are playing a game with a fair die (e.g. no bias for any one side), and your expected payout based on the # you roll is shown the table below. If you have to pay $1 to play, is it worth the cost to play?


|Roll |	Payout|
|-----|-------|
6	|\\$4
5	|\\$2
4	|\\$1
3	|\\$0
2	|\\$0
1	| \\$0

In [13]:
import numpy as np
outcomes = np.repeat(1/6,6)
payouts = np.array([0,0,0,1,2,4])
EX = np.sum(outcomes*payouts) - 1
print(f'expected return is {EX}, so yeah, it\'s still worth to play')

expected return is 0.16666666666666652, so yeah, it's still worth to play


Problem 35
--- 
Suppose you have a dataframe, df, with the following records:

In [14]:
import pandas as pd

raw_data = {'name': ['Willard Morris', 'Al Jennings', 'Omar Mullins', 'Spencer McDaniel'],
                'age': [20, 19, 22, 21],
                'favorite_color': ['blue', 'blue', 'yellow', "green"],
                'grade': [88, 92, 95, 70]}

df = pd.DataFrame(raw_data)
df.head()

,name,age,favorite_color,grade
0,Willard Morris,20,blue,88
1,Al Jennings,19,blue,92
2,Omar Mullins,22,yellow,95
3,Spencer McDaniel,21,green,70


The dataframe is showing information about students. Write code using Python Pandas to select the rows where the students' favorite color is blue or yellow and their grade is above 90.

In [15]:
df.query("favorite_color in ['blue', 'yellow'] and grade > 90")

,name,age,favorite_color,grade
1,Al Jennings,19,blue,92
2,Omar Mullins,22,yellow,95


Problem 34
---
A prime number (or a prime) is a natural number greater than 1 that cannot be formed by multiplying two smaller natural numbers. Given a single #, n, write a function using Python to return whether or not the # is prime. Additionally, if the inputted # is prime, save it into an array, a.

In [39]:
def is_prime(num, a):
    if num < 2:
        return False
    else:
        for i in range(2,num):
            if num%i == 0:
                return False

        a.append(num)
        return True

In [40]:
a = []
is_prime(234293423,a)

False

In [41]:
a

[]

In [42]:
is_prime(2,a)

True

In [43]:
a

[2]

Problem 33
----
Suppose you're consulting for a school district, and the head of the school district thinks the students have above average intelligence. A random sample of thirty students have a mean IQ score of 112. Is there sufficient evidence to support the head's claim?

You can assume the mean IQ score across the population of all students (e.g. including students outside the head's school district) is 100, with a standard deviation of 15.

In [54]:
print(f"""
      hypothesis test:
        null hypothesis H0: mu=0
        alternative hypothesis H1: mu>0
      Assume the alpha is 0.05, corresponding to a z-score of 1.645 (one tail)
      the z score for our sample is (x-mu)/(std/sqrt(N)) = {np.round((112-100)/(15/np.sqrt(30)),3)}
      which is significantly greater than 1.645. 
      Therefore, we can reject the null hypothesis, meaning there's enough evidence to support
      the head's claim
      """)


      hypothesis test:
        null hypothesis H0: mu=0
        alternative hypothesis H1: mu>0
      Assume the alpha is 0.05, corresponding to a z-score of 1.645 (one tail)
      the z score for our sample is (x-mu)/(std/sqrt(N)) = 4.382
      which is significantly greater than 1.645. 
      Therefore, we can reject the null hypothesis, meaning there's enough evidence to support
      the head's claim
      


Problem 32
----

You're analyzing revenue for Company XYZ and want to understand what the revenue per employee was in 2017.

Can you write a query to calculate revenue per employee by product area? Answer will be written in SQL for premium users.



In [16]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE employees (
    employeeid INTEGER,
    product_area VARCHAR(1024),
    compensation INTEGER
    );
    
    CREATE TABLE revenue (
        date VARCHAR(1024),
        product_area VARCHAR(1024),
        revenue INTEGER
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [17]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO employees
    (employeeid, product_area, compensation)
    VALUES
    (1, "Online", 100),
    (2, "Online", 200),
    (3, "In_store", 100),
    (4, "In_store", 100),
    (5, "Online", 100),
    (6, "Online", 100);
    
    INSERT or IGNORE INTO revenue
    (date, product_area, revenue)
    VALUES
    ("2018-01-09", "Online", 40),
    ("2017-03-04", "In_store", 50),
    ("2017-06-03", "Online", 100),
    ("2018-03-04", "In_store", 150),
    ("2017-05-04", "Online", 200),
    ("2017-09-04", "In_store", 10);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [18]:
def get_sql() -> str:
    return """
    
    SELECT e.product_area, 
           SUM(r.revenue) / COUNT(e.employeeid) AS revenue_per_employee
    FROM employees e
    JOIN revenue r
    ON e.product_area = r.product_area
    WHERE r.date >= "2017-01-01" AND
          r.date <= "2017-12-31"
    GROUP BY r.product_area
    ORDER BY 2
    """

crsr.execute(get_sql())
revenue = crsr.fetchall()

import pandas as pd
pd.DataFrame(revenue, columns=['product_area', 'revenue_per_employee'])

,product_area,revenue_per_employee
0,In_store,30
1,Online,150


Problem 31 
---
Suppose you're analyzing a population of 100,000 people, and you're trying to understand life expectancy. Within this population of 100,000 people, 75% can expect to live to the age of 70, while 45% can expect to live to age 80. Given that a person is 70, what is the probability that they live to the age 80?

By using Bayes' theorem, given the person is 70, the probability that he would live up to 80 would be 
P( living to 80 | living to 70) = P( living to 70 | living to 80) * P(living to 80) / P(living to 70)

We already know that P(living to 80) = 0.45 and P(living to 70) = 0.75, also we know that if a person has already lived up to 80, the probability that he's going to live to 70 is 100%. Thus, we just need to plug in the numbers to get the final result. P( living to 80 | living to 70) = (1.0 * 0.45) / 0.75 = 0.6